In [ ]:
from typing import List, Tuple
from model import lstm, mlp, bidirectional_lstm, gru
from utils import read_raw_dataset, select_features_name, read_chunk, train_with_data
from fed_algo import fed_avg
from keras.models import Sequential

In [ ]:
X_raw, y = read_raw_dataset("datasets/pokemon.csv",
                            index_col='id',
                            target_name='MOS',
                            drop=['user_id'])
features = select_features_name(X_raw, y)
X_test, y_test = read_chunk('pokemon', 0, features)
result = {
    'num_clients': [5.0, 20.0, 50.0, 100.0, 150.0],
    'lstm': [],
    'bidirectional': [], 
    'mlp': [], 
    'gru': []
}

In [ ]:
models = [lstm(), bidirectional_lstm(), mlp(), gru()]
models_name = ['lstm', 'bidirectional', 'mlp', 'gru']

In [ ]:
datasets = [
    read_chunk('pokemon', chunk_id + 1, features) for chunk_id in range(5)
]

num_clients = [5, 20, 50, 100, 150]

for i, num_client in enumerate(num_clients):

    num_client_this_round = 5 if i == 0 else num_client - num_clients[i - 1]

    datasets = [
        read_chunk('pokemon',
                   chunk_id + 1 + num_client - num_client_this_round, features)
        for chunk_id in range(num_client_this_round)
    ]

    for name, model in zip(models_name, models):
        old_weights = model.get_weights()
        local_models = [
            train_with_data(model, old_weights, dataset)
            for dataset in datasets
        ]
        new_weights = fed_avg(local_models)
        model.set_weights(new_weights)
        print("Model: {}".format(name))
        result[name].append(model.evaluate(X_test, y_test)[0])

In [ ]:
import json

json_result = json.dumps(result, indent=4)
print(json_result)